# 1. Load Library

In [1]:
!pip install confluent-kafka


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from confluent_kafka import Producer
import json
import os
import time

# 2. Configuration

In [3]:
KAFKA_CONFIG = {
    'bootstrap.servers': 'pkc-921jm.us-east-2.aws.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'WSAS5RJB4L626KGZ',
    'sasl.password': 'pV8yOfEtwP2vyMPgS9FOS+aWhiKZp7Yz63fK152bAqizC8cdVuU59mtKIco1cYZF',
    'client.id': 'reddit-tariff-producer'
}

TOPIC_NAME = 'reddit-tariff-topic'
BATCH_DIRECTORY = r'E:\Users\76044\Desktop\tariff\batches'  # directory where your JSON batch files are saved
FLUSH_INTERVAL = 500  # 500 messages per flush
FLUSH_TIMEOUT = 10    # flush timeout

# 3. Kafka Producer Initialization

In [4]:
producer = Producer(KAFKA_CONFIG)

# 4. Delivery Report Handler

In [5]:
def delivery_report(err, msg):
    if err:
        print(f"❌ Delivery failed: {err}")
    else:
        print(f"✅ Delivered to {msg.topic()} @ offset {msg.offset()}")

# 5. Read and Send Batch Files

In [6]:
msg_counter = 0
for filename in sorted(os.listdir(BATCH_DIRECTORY)):
    if filename.endswith(".json"):
        filepath = os.path.join(BATCH_DIRECTORY, filename)
        print(f"\n📤 Sending file: {filename}")

        with open(filepath, 'r', encoding='utf-8') as f:
            posts = json.load(f)

        for post in posts:
            try:
                producer.produce(
                    topic=TOPIC_NAME,
                    key=post.get("post_id", "no_id"),
                    value=json.dumps(post),
                    callback=delivery_report
                )
                producer.poll(0)
                msg_counter += 1

                if msg_counter % FLUSH_INTERVAL == 0:
                    print(f"🌀 Flushing {FLUSH_INTERVAL} messages...")
                    producer.flush(timeout=FLUSH_TIMEOUT)

            except Exception as e:
                print(f"❌ Error producing message: {e}")


📤 Sending file: batch_001_20250509_150014.json

📤 Sending file: batch_002_20250509_150407.json

📤 Sending file: batch_003_20250509_150753.json

📤 Sending file: batch_004_20250509_151142.json

📤 Sending file: batch_005_20250509_151514.json


# 6. Flush

In [7]:
print("\n🧹 Final flush...")
producer.flush(timeout=30)
print("🎉 All batches sent successfully.")


🧹 Final flush...
✅ Delivered to reddit-tariff-topic @ offset 0
✅ Delivered to reddit-tariff-topic @ offset 1
✅ Delivered to reddit-tariff-topic @ offset 2
✅ Delivered to reddit-tariff-topic @ offset 3
✅ Delivered to reddit-tariff-topic @ offset 4
✅ Delivered to reddit-tariff-topic @ offset 5
✅ Delivered to reddit-tariff-topic @ offset 6
✅ Delivered to reddit-tariff-topic @ offset 7
✅ Delivered to reddit-tariff-topic @ offset 8
✅ Delivered to reddit-tariff-topic @ offset 9
✅ Delivered to reddit-tariff-topic @ offset 10
✅ Delivered to reddit-tariff-topic @ offset 11
✅ Delivered to reddit-tariff-topic @ offset 12
✅ Delivered to reddit-tariff-topic @ offset 13
✅ Delivered to reddit-tariff-topic @ offset 14
✅ Delivered to reddit-tariff-topic @ offset 15
✅ Delivered to reddit-tariff-topic @ offset 16
✅ Delivered to reddit-tariff-topic @ offset 17
✅ Delivered to reddit-tariff-topic @ offset 18
✅ Delivered to reddit-tariff-topic @ offset 19
✅ Delivered to reddit-tariff-topic @ offset 20
✅ Del